### Calling Julia from Python

We are following the tutorial at: [https://blog.esciencecenter.nl/how-to-call-julia-code-from-python-8589a56a98f2](https://blog.esciencecenter.nl/how-to-call-julia-code-from-python-8589a56a98f2)
I printed the page in this [file](/tutorial_clapeyron/call_julia_from_py/tutorial%20instalacion.pdf) in case the link becomes unavailable.

Nevertheless, I will transcribe the most important things just in case the link dies.

Prerequisites:

1 - Python distribution compiled with the shared libpython option. There are workarounds, but this is the most straightforward way.

2 - Julia, the executable that runs the Julia language.

3 - [PyCall](https://github.com/JuliaPy/PyCall.jl), the Julia package that defines the conversions between Julia and Python.
    import Pkg; Pkg.add("PyCall")

4 - PyJulia, the Python package to access Julia from Python.
    pip install julia

It is necessary to create a virtual Julia image from the virtual environment:

python3 -m julia.sysimage sys.so

This will create a sys.so folder in the user directory.

NOTE: If anything in Julia is modified, e.g., adding a package or changing the version of Julia, it is necessary to create sys.so again.

Then, to use Julia from Python,

In [16]:
from julia import Julia

# jl = Julia(sysimage="/home/salvador/sys.so")
jl = Julia(sysimage="../../psrk/sys.so")

The above commands will need to be executed in each session where Julia is 
called; otherwise, PyCall will panic and not know what to do.

In [17]:
# >>> from julia import Base
# >>> Base.julia_cmd()
# out: <PyCall.jlwrap `/PATH/TO/bin/julia-py -Cnative -J/PATH/TO/sys.so -g1`>

from julia import Base

Base.julia_cmd()

<PyCall.jlwrap `/home/salvadorbrandolin/.virtualenvs/psrk/bin/julia-py -Cnative -J/home/salvadorbrandolin/phdpsrk/psrk/sys.so -g1`>

In [18]:
from julia import Main

Main.eval("[x^2 for x in 0:4]")

array([ 0,  1,  4,  9, 16], dtype=int64)

In [19]:
vector = Main.eval("[x^2 for x in 0:4]")

vector

array([ 0,  1,  4,  9, 16], dtype=int64)

In [20]:
Main.vector = Main.eval("[x^2 for x in 0:4]")

In [21]:
Main.vector

array([ 0,  1,  4,  9, 16], dtype=int64)

Importing Clapeyron in Python

In [22]:
from julia import Clapeyron as cp

In [23]:
model1 = cp.PR(["methane"])

In [24]:
vol = cp.volume(model1, 303.15, 101325)

vol

2779.0299445029295

Alternatively, everything can be executed from Julia. In other words, import 
Clapeyron from Julia and run the code directly from there. It seems a bit 
cleaner to me since we have more direct access to the attributes of Julia 
objects. The wrappers generated by PyJulia can be a bit meh... and some 
things are not allowed. For example, if we want to access the critical 
temperatures of the model1 defined above:

In [25]:
model1.params.Tc.values

array([190.564])

In [26]:
%%timeit
cp.volume(model1, 303.15, 101325)

369 µs ± 19.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


You can also execute everything from Julia in the following way:

Anything within Main.eval() is valid Julia code as a string.

In [27]:
Main.eval("using Clapeyron")
Main.model1 = Main.eval('PR(["methane"])')

Main.eval("model1.params.Tc[1]")

190.564

In [28]:
%%timeit
Main.eval("volume(model1, 303.15, 101325)")

150 µs ± 8.28 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
Main.eval("volume(model1, 303.15, 101325)")

2779.0299445029295

This incurs no overhead at all, and the result obtained is:

281 µs ± 24.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)

Note that this result is obtained after running the code several times, so Julia has already started compiling with its JIT.

In any case, it takes three times less time compared to not using Main.eval():

618 µs ± 45.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)

However, it is still noticeable that calling from Main.eval() is faster.

In [30]:
Main.eval('include("../../psrk/julia/mathiascopeman.jl")')

<PyCall.jlwrap α_function>